In [29]:
ZIP_ONLINE = "online_data-20260209T062301Z-1-001.zip"
EXTRACT_DIR = "online_data_unzipped"   # 새로 푼 폴더(기존 것과 섞이지 않게)
LABEL = "fraud"

# 운영 정책
TARGET_RECALL = 0.70
RANDOM_SEED = 42

In [30]:
import os, zipfile, shutil, re
from pathlib import Path

import numpy as np
import pandas as pd

def unzip(zip_path: str, out_dir: str):
    assert os.path.exists(zip_path), f"zip not found: {zip_path}"
    os.makedirs(out_dir, exist_ok=True)
    with zipfile.ZipFile(zip_path, "r") as z:
        z.extractall(out_dir)

def find_dataset_paths(root_dir: str):
    """
    zip 내부 구조를 전혀 가정하지 않고, 파일명/경로에 train/test/check가 포함된 '파일'을 찾는다.
    - 확장자 없어도 OK
    - parquet/arrow/기타여도 일단 후보 수집
    """
    root = Path(root_dir)
    files = [p for p in root.rglob("*") if p.is_file()]
    if not files:
        raise RuntimeError("No files found after unzip.")

    def pick_one(keyword: str):
        cands = []
        for p in files:
            s = str(p).lower()
            # 디렉토리명/파일명 어디든 train/test/check가 들어가면 후보
            if re.search(rf"\b{keyword}\b", s) or f"/{keyword}" in s or f"_{keyword}" in s:
                cands.append(p)
        # 가장 '파일 크기 큰' 후보를 우선 선택(보통 진짜 데이터)
        cands = sorted(cands, key=lambda x: x.stat().st_size, reverse=True)
        return cands[0] if cands else None

    train_p = pick_one("train")
    test_p  = pick_one("test")
    check_p = pick_one("check") or pick_one("val") or pick_one("valid")

    if train_p is None or test_p is None:
        # 디버깅 정보 제공(근거 로그)
        sample = "\n".join([str(p) for p in files[:40]])
        raise RuntimeError(
            "Could not locate train/test files automatically.\n"
            f"Found {len(files)} files. First 40:\n{sample}"
        )

    return str(train_p), str(test_p), (str(check_p) if check_p else None)

# 실행
# (기존에 풀어둔 폴더가 있더라도, 혼선 방지 위해 새 폴더에 풀어버림)
if os.path.exists(EXTRACT_DIR):
    shutil.rmtree(EXTRACT_DIR)

unzip(ZIP_ONLINE, EXTRACT_DIR)
train_path, test_path, check_path = find_dataset_paths(EXTRACT_DIR)

print("Picked paths:")
print(" train:", train_path)
print(" test :", test_path)
print(" check:", check_path)


Picked paths:
 train: online_data_unzipped/online_data/train
 test : online_data_unzipped/online_data/test
 check: online_data_unzipped/online_data/check


In [31]:
def read_parquet_flexible(path: str) -> pd.DataFrame:
    """
    - path가 확장자 없어서 pd.read_parquet가 실패하는 경우가 있어, 임시로 .parquet 복사본 만들어 재시도
    """
    try:
        return pd.read_parquet(path)
    except Exception:
        tmp = path + ".parquet"
        if not os.path.exists(tmp):
            shutil.copy(path, tmp)
        return pd.read_parquet(tmp)

df_train = read_parquet_flexible(train_path)
df_test  = read_parquet_flexible(test_path)
df_check = read_parquet_flexible(check_path) if check_path else None

print("Shapes:", df_train.shape, df_test.shape, None if df_check is None else df_check.shape)
assert LABEL in df_train.columns and LABEL in df_test.columns, f"Missing label '{LABEL}'"


Shapes: (609655, 60) (114209, 60) (166904, 60)


In [32]:
def split_report(df: pd.DataFrame, name: str):
    y = df[LABEL].astype(int)
    return {
        "split": name,
        "rows": int(df.shape[0]),
        "cols": int(df.shape[1]),
        "pos_cnt": int(y.sum()),
        "pos_rate": float(y.mean()),
    }

rep = [split_report(df_train, "train"), split_report(df_test, "test")]
if df_check is not None:
    rep.append(split_report(df_check, "check"))
display(pd.DataFrame(rep))

,split,rows,cols,pos_cnt,pos_rate
0,train,609655,60,6598,0.010823
1,test,114209,60,2096,0.018352
2,check,166904,60,0,0.000000


In [33]:
from sklearn.model_selection import train_test_split

DROP_IF_EXISTS = [
    "client_id", "card_id", "transaction_id", "id",  # 흔한 키
]

def normalize_schema(train: pd.DataFrame, test: pd.DataFrame, check: pd.DataFrame | None):
    """
    1) 공통 컬럼만 사용 (버전 차이/누락 방지)
    2) label 보존
    """
    cols = set(train.columns) & set(test.columns)
    if check is not None:
        cols = cols & set(check.columns)
    cols = sorted(cols)

    train2 = train[cols].copy()
    test2  = test[cols].copy()
    check2 = check[cols].copy() if check is not None else None
    return train2, test2, check2

df_train2, df_test2, df_check2 = normalize_schema(df_train, df_test, df_check)

def separate_xy(df: pd.DataFrame):
    y = df[LABEL].astype(int).values
    X = df.drop(columns=[LABEL], errors="ignore").copy()
    return X, y

X_train, y_train = separate_xy(df_train2)
X_test,  y_test  = separate_xy(df_test2)
X_check, y_check = (separate_xy(df_check2) if df_check2 is not None else (None, None))

# ID/키 컬럼 제거(있으면)
for c in DROP_IF_EXISTS:
    if c in X_train.columns:
        X_train.drop(columns=[c], inplace=True)
        X_test.drop(columns=[c], inplace=True)
        if X_check is not None:
            X_check.drop(columns=[c], inplace=True)

# datetime 컬럼 자동 제거(있으면)
dt_cols = list(X_train.select_dtypes(include=["datetime64[ns]", "datetime64[ns, UTC]"]).columns)
if dt_cols:
    X_train.drop(columns=dt_cols, inplace=True, errors="ignore")
    X_test.drop(columns=dt_cols, inplace=True, errors="ignore")
    if X_check is not None:
        X_check.drop(columns=dt_cols, inplace=True, errors="ignore")

# object 중 "숫자처럼 생긴 문자열"은 numeric으로 강제 변환
def coerce_object_to_numeric(df: pd.DataFrame):
    obj_cols = list(df.select_dtypes(include=["object"]).columns)
    for c in obj_cols:
        # 변환 시도
        s = pd.to_numeric(df[c], errors="coerce")
        # 숫자화 성공 비율이 충분히 높으면 numeric으로 전환
        ok_ratio = s.notna().mean()
        if ok_ratio >= 0.95:   # 기준은 보수적으로
            df[c] = s
    return df

X_train = coerce_object_to_numeric(X_train)
X_test  = coerce_object_to_numeric(X_test)
if X_check is not None:
    X_check = coerce_object_to_numeric(X_check)

# 최종 공통 컬럼 재강제(위 드랍으로 차이 생기면 방지)
common_cols = sorted(set(X_train.columns) & set(X_test.columns) & (set(X_check.columns) if X_check is not None else set(X_train.columns)))
X_train = X_train[common_cols]
X_test  = X_test[common_cols]
if X_check is not None:
    X_check = X_check[common_cols]

print("Final feature cols:", len(common_cols))
print("Datetime dropped:", dt_cols)


Final feature cols: 56
Datetime dropped: ['date']


In [34]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

def build_preprocess(X: pd.DataFrame):
    num_cols = list(X.select_dtypes(include=["number", "bool"]).columns)
    cat_cols = [c for c in X.columns if c not in num_cols]

    numeric_tf = Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
    ])

    categorical_tf = Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("ohe", OneHotEncoder(handle_unknown="ignore", sparse_output=True)),
    ])

    preprocess = ColumnTransformer(
        transformers=[
            ("num", numeric_tf, num_cols),
            ("cat", categorical_tf, cat_cols),
        ],
        remainder="drop"
    )
    return preprocess, num_cols, cat_cols

preprocess, num_cols, cat_cols = build_preprocess(X_train)
print("Numeric:", len(num_cols), "Categorical:", len(cat_cols))

Numeric: 56 Categorical: 0


In [35]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV

def build_models(preprocess):
    models = {}

    # 1) 팀원 baseline 핵심: cost-sensitive logistic
    models["logit_l2_balanced"] = Pipeline([
        ("prep", preprocess),
        ("scaler", StandardScaler(with_mean=False)),  # sparse 대응
        ("clf", LogisticRegression(max_iter=500, n_jobs=-1, class_weight="balanced"))
    ])

    # 2) HGB: 빠르고 강력한 비선형 기준점
    models["hgb"] = Pipeline([
        ("prep", preprocess),
        ("clf", HistGradientBoostingClassifier(random_state=RANDOM_SEED))
    ])

    # 3) Linear SVM + Calibration: 선형 경계 + 확률 (운영형 threshold에 유리)
    base_svm = Pipeline([
        ("prep", preprocess),
        ("scaler", StandardScaler(with_mean=False)),
        ("svc", LinearSVC(class_weight="balanced", random_state=RANDOM_SEED))
    ])
    models["linear_svm_calibrated"] = CalibratedClassifierCV(base_svm, method="sigmoid", cv=3)

    # 4~5) LGBM/XGB는 설치돼 있을 때만 추가(없으면 자동 제외)
    try:
        from lightgbm import LGBMClassifier
        models["lgbm"] = Pipeline([
            ("prep", preprocess),
            ("clf", LGBMClassifier(
                n_estimators=800, num_leaves=63, learning_rate=0.05,
                subsample=0.8, colsample_bytree=0.8,
                n_jobs=-1, random_state=RANDOM_SEED,
                objective="binary", class_weight="balanced"
            ))
        ])
    except Exception:
        pass

    try:
        from xgboost import XGBClassifier
        spw = (y_train==0).sum() / max((y_train==1).sum(), 1)
        models["xgb"] = Pipeline([
            ("prep", preprocess),
            ("clf", XGBClassifier(
                n_estimators=600, max_depth=6, learning_rate=0.05,
                subsample=0.8, colsample_bytree=0.8,
                tree_method="hist", n_jobs=-1, random_state=RANDOM_SEED,
                objective="binary:logistic", eval_metric="aucpr",
                scale_pos_weight=spw
            ))
        ])
    except Exception:
        pass

    return models

models = build_models(preprocess)
print("Models:", list(models.keys()))


Models: ['logit_l2_balanced', 'hgb', 'linear_svm_calibrated', 'lgbm', 'xgb']


In [36]:
from sklearn.metrics import average_precision_score, roc_auc_score, precision_recall_curve, confusion_matrix

def score_proba_or_margin(model, X):
    if hasattr(model, "predict_proba"):
        return model.predict_proba(X)[:, 1]
    if hasattr(model, "decision_function"):
        raw = model.decision_function(X)
        # margin을 [0,1]로 정규화(확률은 아니지만 threshold sweep엔 충분)
        return (raw - raw.min()) / (raw.max() - raw.min() + 1e-12)
    raise RuntimeError("Model has neither predict_proba nor decision_function")

def choose_threshold_under_recall(y_true, y_score, target_recall=0.70):
    precision, recall, thr = precision_recall_curve(y_true, y_score)
    # precision_recall_curve는 thr 길이가 하나 짧음
    p, r, t = precision[1:], recall[1:], thr

    ok = r >= target_recall
    if not np.any(ok):
        i = int(np.argmax(r))
        return float(t[i]), float(p[i]), float(r[i]), "target_recall_not_met"

    cand = np.where(ok)[0]
    best_p = np.max(p[cand])
    bests = cand[p[cand] == best_p]
    i = int(bests[np.argmax(t[bests])])  # 같은 precision이면 threshold 큰 것(=경보량↓)
    return float(t[i]), float(p[i]), float(r[i]), "picked_best_precision_under_recall"

def ops_at_threshold(y_true, y_score, thr):
    y_hat = (y_score >= thr).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_true, y_hat).ravel()
    prec = tp / (tp + fp + 1e-12)
    rec  = tp / (tp + fn + 1e-12)
    return {
        "precision": float(prec),
        "recall": float(rec),
        "alert_rate": float(y_hat.mean()),
        "tp": int(tp), "fp": int(fp), "fn": int(fn), "tn": int(tn)
    }

results = []
run_log = {
    "data_paths": {"train": train_path, "test": test_path, "check": check_path},
    "policy": {"target_recall": TARGET_RECALL},
    "feature_policy": {
        "dropped_ids": [c for c in DROP_IF_EXISTS if c in df_train.columns],
        "dropped_datetime_cols": dt_cols,
        "final_feature_count": len(common_cols),
        "numeric_count": len(num_cols),
        "categorical_count": len(cat_cols),
    }
}

for name, model in models.items():
    print(f"\n=== {name} ===")
    model.fit(X_train, y_train)

    s_test = score_proba_or_margin(model, X_test)
    test_pr  = float(average_precision_score(y_test, s_test))
    test_roc = float(roc_auc_score(y_test, s_test))

    thr, thr_p, thr_r, note = choose_threshold_under_recall(y_test, s_test, TARGET_RECALL)
    test_ops = ops_at_threshold(y_test, s_test, thr)

    # check는 운영 시뮬레이션(양성이 0개일 수 있음 -> 그 자체가 중요한 사실)
    check_ops = None
    check_pos = None
    if X_check is not None:
        s_check = score_proba_or_margin(model, X_check)
        check_pos = int(np.sum(y_check == 1))
        # check에 양성이 0개면 recall/PR은 해석 불가 -> alert_rate 중심으로 모니터링
        check_ops = ops_at_threshold(y_check, s_check, thr)

    print(f"TEST  PR-AUC={test_pr:.4f} ROC-AUC={test_roc:.4f} | thr={thr:.6f} ({note})")
    print("TEST  ops:", test_ops)
    if check_ops is not None:
        print("CHECK ops:", check_ops, "| check_pos:", check_pos)

    results.append({
        "model": name,
        "test_pr_auc": test_pr,
        "test_roc_auc": test_roc,
        "thr": thr,
        "test_precision@thr": test_ops["precision"],
        "test_recall@thr": test_ops["recall"],
        "test_alert_rate@thr": test_ops["alert_rate"],
        "test_fp": test_ops["fp"],
        "test_fn": test_ops["fn"],
        "check_alert_rate@thr": (check_ops["alert_rate"] if check_ops is not None else np.nan),
        "check_pos_cnt": (check_pos if check_pos is not None else np.nan),
    })

summary = pd.DataFrame(results).sort_values(
    ["test_pr_auc", "test_precision@thr"], ascending=False
).reset_index(drop=True)

display(summary)



=== logit_l2_balanced ===
TEST  PR-AUC=0.2382 ROC-AUC=0.8819 | thr=0.664233 (picked_best_precision_under_recall)
TEST  ops: {'precision': 0.10558872185859165, 'recall': 0.7003816793893127, 'alert_rate': 0.12173296325158263, 'tp': 1468, 'fp': 12435, 'fn': 628, 'tn': 99678}
CHECK ops: {'precision': 0.0, 'recall': 0.0, 'alert_rate': 0.11002732109476106, 'tp': 0, 'fp': 18364, 'fn': 0, 'tn': 148540} | check_pos: 0

=== hgb ===
TEST  PR-AUC=0.8715 ROC-AUC=0.9855 | thr=0.679801 (picked_best_precision_under_recall)
TEST  ops: {'precision': 0.9780292942743003, 'recall': 0.7008587786259539, 'alert_rate': 0.013151327828805086, 'tp': 1469, 'fp': 33, 'fn': 627, 'tn': 112080}
CHECK ops: {'precision': 0.0, 'recall': 0.0, 'alert_rate': 0.00034151368451325315, 'tp': 0, 'fp': 57, 'fn': 0, 'tn': 166847} | check_pos: 0

=== linear_svm_calibrated ===
TEST  PR-AUC=0.2029 ROC-AUC=0.8837 | thr=0.025359 (picked_best_precision_under_recall)
TEST  ops: {'precision': 0.1065032987747408, 'recall': 0.7008587786259

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


TEST  PR-AUC=0.9102 ROC-AUC=0.9896 | thr=0.924202 (picked_best_precision_under_recall)
TEST  ops: {'precision': 0.9945799457994574, 'recall': 0.7003816793893127, 'alert_rate': 0.012923675016855063, 'tp': 1468, 'fp': 8, 'fn': 628, 'tn': 112105}
CHECK ops: {'precision': 0.0, 'recall': 0.0, 'alert_rate': 2.3965872597421273e-05, 'tp': 0, 'fp': 4, 'fn': 0, 'tn': 166900} | check_pos: 0

=== xgb ===
TEST  PR-AUC=0.8582 ROC-AUC=0.9868 | thr=0.850684 (picked_best_precision_under_recall)
TEST  ops: {'precision': 0.9130974549968959, 'recall': 0.7018129770992363, 'alert_rate': 0.014105718463518636, 'tp': 1471, 'fp': 140, 'fn': 625, 'tn': 111973}
CHECK ops: {'precision': 0.0, 'recall': 0.0, 'alert_rate': 0.0007549249868187701, 'tp': 0, 'fp': 126, 'fn': 0, 'tn': 166778} | check_pos: 0


,model,test_pr_auc,test_roc_auc,thr,test_precision@thr,test_recall@thr,test_alert_rate@thr,test_fp,test_fn,check_alert_rate@thr,check_pos_cnt
0,lgbm,0.910221,0.989567,0.924202,0.994580,0.700382,0.012924,8,628,0.000024,0
1,hgb,0.871526,0.985462,0.679801,0.978029,0.700859,0.013151,33,627,0.000342,0
2,xgb,0.858233,0.986790,0.850684,0.913097,0.701813,0.014106,140,625,0.000755,0
3,logit_l2_balanced,0.238226,0.881921,0.664233,0.105589,0.700382,0.121733,12435,628,0.110027,0
4,linear_svm_calibrated,0.202855,0.883685,0.025359,0.106503,0.700859,0.120770,12324,627,0.108637,0


좋아. 큰 틀에서 계산 로직이 “깨졌다”거나 결과가 불가능한 형태는 아니고, 지금 출력은 운영형 Stage1 스크리닝 관점에서 해석 가능한 상태야.
다만 “이상하게 좋은” 수치가 몇 개 있고, 그건 데이터/피처 누수(leakage) 또는 평가 설계 착시 가능성이 있어서 확인해야 할 체크포인트가 명확히 있다. 아래에 (A) 잘못/위험 신호 점검 → (B) 결과 해석 순서로 정리한다.

A. “잘못된 부분 없는지” 점검 (핵심 리스크 5개)
1) CHECK에서 check_pos_cnt = 0 (사기 1이 없음)

출력상 check는 fraud=1이 0건이야.

그래서 check에서 precision/recall이 0으로 찍히는 건 정상이고, **check는 성능 평가용이 아니라 “경보율(alert_rate) 모니터링용”**으로만 의미가 있어.

✅ 결론: 이건 오류가 아니라 데이터 특성.
⚠️ 단, check로 “드리프트/운영부담”만 봐야 한다.

2) LGBM/HGB가 precision≈0.99인데 FP=8~33 (너무 좋음)

LGBM: recall≈0.700, TP=1468, FP=8 → precision 0.994

HGB: recall≈0.701, TP=1469, FP=33 → precision 0.978

이게 “절대 불가능”은 아니지만, 현실 사기탐지에서는 매우 비정상적으로 좋다고 보는 게 맞다.
보통 이런 급의 성능은 아래 중 하나일 때 나온다:

(a) 테스트셋이 “쉽게 구분되는” 분포 (사기 패턴이 너무 분리됨)

(b) 피처 누수(레이블/미래정보가 피처에 섞임)

(c) 전처리/평가에서 train과 test가 섞임 (time split 깨짐, join 문제 등)

✅ 지금 당장 해야 할 1차 검증:

타깃 컬럼/파생 피처에 fraud, label, is_fraud, chargeback, dispute 류가 포함돼 있는지

시간 파생이 “미래”를 보고 만든 rolling feature인지 (예: 하루 단위 집계가 미래 거래 포함)

train/test/check가 서로 겹치는 key(거래ID 등)로 중복이 있는지

이 부분을 확인하지 않고 “모델이 압도적으로 좋다”라고 결론 내리면, 강사 질문에 바로 털린다.

3) Logit/SVM의 alert_rate가 0.12인데 트리들은 0.013대

logit: alert_rate≈0.122, precision≈0.106

svm: alert_rate≈0.121, precision≈0.106

hgb/lgbm: alert_rate≈0.013, precision≈0.98~0.99

같은 Recall constraint(≈0.70)를 만족시키면서 경보율이 10배 차이 나는 건, 트리 모델이 훨씬 “선별”을 잘 했다는 뜻이기도 하지만, 동시에 위 2)처럼 누수 의심을 더 키우는 신호야.

✅ 결론: 로직 자체는 정상이나, 데이터/피처가 너무 강한 신호를 갖고 있을 가능성이 커서 검증 필요.

4) LGBM 경고: “X does not have valid feature names…”

이건 scikit-learn wrapper에서 흔한 경고.

전처리 결과가 numpy/희소행렬로 변하면서 feature name이 사라져서 뜨는 메시지.

✅ 결론: 치명적 오류 아님. 결과는 유효.

5) Threshold 값들이 “극단적으로 큼/작음”은 정상

logit thr≈0.664

lgbm thr≈0.924

svm thr≈0.025

너희 정책이 “Recall≥0.70 만족하는 구간 중 Precision 최대”라서, 모델별 score scale이 달라 threshold는 비교 대상이 아니다.
비교는 PR-AUC/ROC-AUC/alert_rate/FP/FN로 한다.

✅ 결론: threshold 자체가 이상한 건 아님.

B. 결과 해석 (Stage1 운영 관점)
1) 성능 요약(테스트 기준)

표가 말하는 결론은 명확함:

1등: LGBM

PR-AUC 0.910, ROC-AUC 0.990

Recall≈0.700 유지하면서 FP=8, Alert rate≈1.29%

운영적으로 “조사량 적고 놓침도 제한”이라 최상

2등: HGB

PR-AUC 0.872, ROC-AUC 0.985

FP=33, Alert rate≈1.32%

LGBM보다 약간 거칠지만 여전히 매우 강함

3등: XGB

PR-AUC 0.858, ROC-AUC 0.987

FP=140, Alert rate≈1.41%

트리 중에선 FP가 상대적으로 큼

하위: Logistic / Calibrated SVM

PR-AUC 0.23/0.20대로 낮고,

Recall≈0.70 맞추려면 alert_rate가 12%대로 터짐(=운영부담 큼)

즉 Stage1 스크리닝 정책에는 부적합(이 데이터/피처 기준)

2) check 해석(중요)

check는 fraud가 0이라 성능을 논할 수 없고,

LGBM check_alert_rate ≈ 0.000024 (0.0024%)

HGB ≈ 0.000342

XGB ≈ 0.000755

Logit/SVM은 0.11대

여기서 의미는:

트리 모델은 check 구간에서 거의 경보를 안 울림 → 운영부담 거의 없음

반면 logit/svm은 check에서 11% 경보 → 운영에서 바로 장애 수준

다만! check에 fraud가 0이면 “조용한 게 좋은지” 판단이 어렵다.

실제로 fraud가 없어 조용한 걸 수도 있고,

분포가 바뀌어서 점수가 내려가 조용한 걸 수도 있다.

그래서 check는 다음을 같이 봐야 한다:

score 분포(평균/상위 1% 분위) 변화

alert_rate의 시간별 추세

결론: “잘못은 없지만, 너무 좋아서 검증이 필요”

계산/출력은 정상.

하지만 LGBM/HGB 성능이 너무 강력해서 ‘피처 누수/분할/중복’ 검증 없이 발표하면 위험.

너한테 바로 필요한 “필수 검증 3종 세트” (다음 셀로 실행 권장)

label 누수 컬럼 탐지

컬럼명에 fraud|label|target|chargeback|dispute|cbk|refund_after 같은 패턴 포함 여부

train/test/check 중복 검사

transaction_id 같은 키가 있으면 split 간 중복 row 존재 여부

시간 누수 검사

rolling/window 파생이 “현재 시점 이후 거래”를 포함해 계산됐는지(가능하면 생성 코드/정의 확인)

원하면, 너가 쓰는 데이터 컬럼 이름 기준으로 위 3개를 바로 찍는 진단 코드를 “셀 1개짜리”로 만들어 줄게.